In [ ]:
# prepare dataset
# load pretrained tokenizer, call it with dataset
# build PyTorch dataset with encodings
# load pretrained model
# load trainer and train int / native pytorch trianing loop

In [1]:
!pip install transformers==4.29.0
!pip install git+https://github.com/huggingface/accelerate
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW

  Cloning https://github.com/huggingface/accelerate to c:\users\ameen\appdata\local\temp\pip-req-build-i1bb8yok


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
import pandas as pd
from tqdm import tqdm


# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
batch_size = 16
max_input_length = 128
max_output_length = 128
learning_rate = 1e-5
num_epochs = 2

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("K:\Github\ActasGPT\model\chatgpt-prompts")
model = AutoModelForSeq2SeqLM.from_pretrained("K:\Github\ActasGPT\model\chatgpt-prompts", from_tf=True)

# Move model to GPU device
model.to(device)

# Load the dataset
df = pd.read_csv("K:\Github\ActasGPT\model\prompts.csv")

# Define the dataset class
class ChatGPTDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_length, max_output_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_text = self.data['act'][index]
        target_text = self.data['prompt'][index]

        input_encoding = self.tokenizer.batch_encode_plus(
            [input_text],
            max_length=self.max_input_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        target_encoding = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.max_output_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = input_encoding['input_ids'].squeeze().to(device)
        input_attention_mask = input_encoding['attention_mask'].squeeze().to(device)
        target_ids = target_encoding['input_ids'].squeeze().to(device)
        target_attention_mask = target_encoding['attention_mask'].squeeze().to(device)

        return {
            'input_ids': input_ids,
            'attention_mask': input_attention_mask,
            'target_ids': target_ids,
            'target_attention_mask': target_attention_mask
        }

# Prepare the dataset
dataset = ChatGPTDataset(df, tokenizer, max_input_length, max_output_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Prepare the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in progress_bar:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        target_ids = batch['target_ids']
        target_attention_mask = batch['target_attention_mask']

        optimizer.zero_grad()

        inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': target_ids,
            'decoder_attention_mask': target_attention_mask
        }

        # Move inputs to GPU device
        inputs = {key: value.to(device) for key, value in inputs.items()}

        outputs = model(**inputs)

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss}")

# Because of the large size of the model, the Jupyter kernel will continue running the cell and generate the output even if the notebook is closed, but it won't display the results.    
model.save_pretrained('chatgpt-prompts')
tokenizer.save_pretrained('chatgpt-prompts')

All TF 2.0 model weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
C:\Users\ameen\anaconda3\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/2:  45%|████▌     | 5/11 [2:33:16<3:02:23, 1823.91s/it, Loss=5.49]